In [ ]:
# import the libraries needed to represent the tsp and to embed it on the quantum annealer
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from neal import SimulatedAnnealingSampler

import numpy as np
import dimod
import itertools 
import minorminer
import networkx as nx
import dwave_networkx as dnx


In [ ]:
# problem parameters
PROBLEM_NAME = 'tsp1.txt'
TOKEN = ''
D = np.loadtxt('Test_Problems/' + PROBLEM_NAME, dtype='i', delimiter=' ')

n = D.shape[0]
num_reads = 100

print(f'D is: \n{D}')

In [ ]:
# normalization factors
A_NORMALIZATION = np.max(D) + 1
B_NORMALIZATION = 1

In [ ]:
from pyqubo import Binary, Constraint, Array

x = Array.create('x', (n,n), 'BINARY')
x

In [ ]:
H_a1 = 0
for v in range(n):
    H_temp = 1
    for j in range(n):
        H_temp -= x[v, j]

    H_a1 += H_temp * H_temp

H_a2 = 0
for j in range(n):
    H_temp = 1
    for v in range(n):
        H_temp -= x[v, j]

    H_a2 += H_temp * H_temp

H_a3 = 0
for u in range(n):
    for v in range(n):
            k = 1
            for j in range(n):
                H_a3 +=  x[u, j] * x[v, k]

                k += 1
                if k == n: k = 0


In [ ]:
H_b = 0

for u in range(n):
    for v in range(n):
        if u != v:
            k = 1
            for j in range(n):
                H_b +=  D[u,v] * x[u, j] * x[v, k]

                k += 1
                if k == n: k = 0

H_b

In [ ]:
H_A = A_NORMALIZATION * H_a1 + A_NORMALIZATION * H_a2 #+ A_NORMALIZATION * H_a3
H_B = B_NORMALIZATION * H_b

H = H_A + H_B

model = H.compile()
qubo, _= model.to_qubo()

BQM = dimod.BinaryQuadraticModel.from_qubo(qubo)

Util functions needed to visualize the results, both in simulated annealing and on the real QPU

In [ ]:
def print_response_data(response):
    pos_sets = []
    response = response.lowest()
    # ------- Print results to user -------
    print('-' * 100)
    print('{:>20s}{:>42s}{:>22s}'.format('Set 1','Energy',"Count"))
    print('-' * 100)
    for sample, E, occ in response.data(fields=['sample','energy',"num_occurrences"]):
        S0 = [k for k,v in sample.items() if v == 0]
        S1 = [k for k,v in sample.items() if v == 1]
        pos_sets.append(S1)
        print('{:>30s}{:^30s}{:^15s}'.format(str(S1),str(E),str(occ)))
    
    return pos_sets

def map_variables(pos_set):
    m_set = []
    for i in range(len(pos_set)):
        x = pos_set[i].replace(']', '').split('[')
        m_set.append([int(x[1]), int(x[2])])
    
    return m_set

def return_solution(pos_solution):
    for p_set in pos_solution:
        m_set = map_variables(p_set)
        s_res = sorted(m_set, key=lambda x: x[1])
        for i in range(len(s_res) - 1):
            print((s_res[i])[0], end='-->')
        
        print((s_res[len(s_res) - 1])[0])

## Solve the problem with Simulated Annealing

In [ ]:
sampler = SimulatedAnnealingSampler()

response_SA = sampler.sample(BQM, num_reads = num_reads)
response_SA.aggregate()

In [ ]:
solution_SA = print_response_data(response_SA.aggregate())

In [ ]:
print(f'There are {len(solution_SA)} solutions')
return_solution(solution_SA)

## Solve the problem on the QPU

In [ ]:
sampler = EmbeddingComposite(DWaveSampler(token=TOKEN))

response_QPU = sampler.sample(BQM, num_reads=num_reads, label='Test tsp all ones')
response_QPU.aggregate()

In [ ]:
solution_QPU = print_response_data(response_QPU.aggregate())

In [ ]:
print(f'There are {len(solution_QPU)} solutions')
return_solution(solution_QPU)

## Number of required qubits

In [ ]:
# identify the embedded graph on the architecture of the solver used in the real annealing

device = DWaveSampler(token=TOKEN)
device.solver.data['id']

QUBO_graph = BQM.to_networkx_graph()
QPU_graph = device.solver.data['properties']["couplers"]

embedded_graph = minorminer.find_embedding(QUBO_graph.edges(), QPU_graph)

embedded_graph

In [ ]:
# print the number of qubit required in the embedding and the minimal and maximal length of the chains created

sublist = [values for keys, values in embedded_graph.items()]
flat_list = set(itertools.chain(*sublist))    

max_chain_length = None
min_chain_length = None

for _, chain in embedded_graph.items():
    if max_chain_length is None:
        max_chain_length = len(chain)
        min_chain_length = len(chain)

    if len(chain) > max_chain_length:
        max_chain_length = len(chain)

    if len(chain) < min_chain_length:
        min_chain_length = len(chain)

    
print("Embedding requires {} qubits and has chain lengths between {}-{}".format(len(flat_list),min_chain_length, max_chain_length))